
## Populate Database

In [ ]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

In [ ]:


user='martin_svitek'
password='T67wni047Ugy2uUx'

In [2]:
# Conection
host = 'localhost'
port = '5433'
database = 'layereddb'
schema='berlin_source_data'

#connection to db after you opened tunnel
engine = create_engine(f'postgresql+psycopg2://{user_name}:{password}@{host}:{port}/{database}')

NameError: name 'create_engine' is not defined

In [3]:
#this is where you create table with constraints and references first
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.test_table_{user_name} (
    id INTEGER PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    score INTEGER
);
"""

# Execute the query to create empty table
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()  # commit the transaction

NameError: name 'engine' is not defined

In [4]:
#create vet clinics table if not exists
CREATE TABLE IF NOT EXISTS veterinary_clinics (
  clinic_id           VARCHAR(64) PRIMARY KEY,              -- e.g. 'node/123' or your stringified OSM id
  clinic_name         VARCHAR(200) NOT NULL,
  street              VARCHAR(100) NOT NULL,
  house_number        VARCHAR(10)  NOT NULL,
  postcode            VARCHAR(5),
  city                VARCHAR(100) NOT NULL,

  district_id         INT REFERENCES districts(district_id) ON UPDATE CASCADE ON DELETE SET NULL,
  neighbourhood_id    INT REFERENCES neighbourhoods(neighbourhood_id) ON UPDATE CASCADE ON DELETE SET NULL,

  phone_number        VARCHAR(50),
  website             VARCHAR(100),
  email               VARCHAR(100),
  opening_hours       VARCHAR(255),
  operator            VARCHAR(100),
  speciality          VARCHAR(100),
  wheelchair_acces    VARCHAR(50),                          -- normalized: yes|no|limited
  full_address        VARCHAR(255),

  -- geometries (SRID 4326)
  pt_geom             geometry(Point, 4326),
  footprint_geom      geometry(Geometry, 4326),             -- Polygon/MultiPolygon if you have it

 
);

-- If you already had a table but were missing columns, add them idempotently:
ALTER TABLE veterinary_clinics
  ADD COLUMN IF NOT EXISTS speciality          VARCHAR(100),
  ADD COLUMN IF NOT EXISTS wheelchair_acces    VARCHAR(50),
  ADD COLUMN IF NOT EXISTS full_address        VARCHAR(255),
  ADD COLUMN IF NOT EXISTS pt_geom             geometry(Point, 4326),
  ADD COLUMN IF NOT EXISTS footprint_geom      geometry(Geometry, 4326),
  ADD COLUMN IF NOT EXISTS created_at          TIMESTAMP WITH TIME ZONE DEFAULT NOW(),
  ADD COLUMN IF NOT EXISTS updated_at          TIMESTAMP WITH TIME ZONE DEFAULT NOW();

-- Helpful constraints
ALTER TABLE veterinary_clinics
  ADD CONSTRAINT IF NOT EXISTS chk_postcode_len
    CHECK (postcode IS NULL OR char_length(postcode) = 5),
  ADD CONSTRAINT IF NOT EXISTS chk_wheelchair
    CHECK (wheelchair_acces IN ('yes','no','limited') OR wheelchair_acces IS NULL);

-- De-dup safeguard: if you consider (clinic_name, street, house_number, postcode) unique
CREATE UNIQUE INDEX IF NOT EXISTS uq_clinic_name_addr
ON veterinary_clinics (lower(clinic_name), lower(street), lower(house_number), postcode);

-- Indexes for performance
CREATE INDEX IF NOT EXISTS clinics_district_idx     ON veterinary_clinics (district_id);
CREATE INDEX IF NOT EXISTS clinics_neighbour_idx    ON veterinary_clinics (neighbourhood_id);
CREATE INDEX IF NOT EXISTS clinics_pt_gix           ON veterinary_clinics USING GIST (pt_geom);
CREATE INDEX IF NOT EXISTS clinics_footprint_gix    ON veterinary_clinics USING GIST (footprint_geom);
CREATE INDEX IF NOT EXISTS clinics_name_trgm_idx    ON veterinary_clinics USING GIN (clinic_name gin_trgm_ops);
CREATE INDEX IF NOT EXISTS clinics_address_trgm_idx ON veterinary_clinics USING GIN (full_address gin_trgm_ops);

-- Auto-update updated_at
CREATE OR REPLACE FUNCTION set_updated_at()
RETURNS TRIGGER AS $$
BEGIN
  NEW.updated_at := NOW();
  RETURN NEW;
END; $$ LANGUAGE plpgsql;

DROP TRIGGER IF EXISTS trg_clinics_updated_at ON veterinary_clinics;
CREATE TRIGGER trg_clinics_updated_at
BEFORE UPDATE ON veterinary_clinics
FOR EACH ROW EXECUTE FUNCTION set_updated_at();



SyntaxError: invalid syntax (2366030662.py, line 2)